<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

# Выбор локации для скважины

Добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.
Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. **Цель проекта - построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.** 

## Загрузка и подготовка данных

Установка библиотеки для исследовательского анализа

In [4]:
pip install sweetviz

     |████████████████████████████████| 15.1 MB 2.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
# импорт всех библиотек и инструментов
import pandas as pd
import matplotlib.pyplot as plt
import sweetviz as sv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [10]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

Создадим список с датафреймами для последующей работы в циклах и выведем на экран первые строки таблиц.

In [11]:
datas = [df_0, df_1, df_2]

In [12]:
for i in datas:
    display(i.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


При помощи библиотеки `sweetviz` (установлена вручную вначале документа) проведем исследовательский анализ и выведем три отчета для каждого датафрейма.

In [13]:
for i in datas:
    report = sv.analyze(i)
    report.show_html()

                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


**ВЫВОД**

Пропущенных значений нет, дубликатов нет. Все данные относительно однородны. Можно приступать к обучению моделей.

## Обучение и проверка модели

Чтобы не дублировать код для каждого региона, воспользуемся списком Датафреймов и в цикле выведем для каждого региона средний запас предсказанного сырья, а также RMSE модели.

In [14]:
for index, i in enumerate(datas):
    features = i.drop(['id', 'product'], axis=1) # в признаки выделяем столбцы 'f0', 'f1', 'f2'
    target = i['product'] # в целевой признак выделяем столбец 'product'
    
    # отделяем 25% данных для валидационной выборки
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)

    model = LinearRegression() # создаем модель линенйной регрессии
    model.fit(features_train, target_train) # обучаем модель
    predictions = model.predict(features_valid) # предсказания
    answers = target_valid # правильные ответы
    rmse = mean_squared_error(answers, predictions) ** 0.5 # расчитываем rmse
    print('Средний запас предсказанного сырья в регионе №', index+1, ':', predictions.mean().round(3), 
            '\nRMSE модели на валидационной выборке:', rmse.round(3),
          '\n')

Средний запас предсказанного сырья в регионе № 1 : 92.593 
RMSE модели на валидационной выборке: 37.579 

Средний запас предсказанного сырья в регионе № 2 : 68.729 
RMSE модели на валидационной выборке: 0.893 

Средний запас предсказанного сырья в регионе № 3 : 94.965 
RMSE модели на валидационной выборке: 40.03 



**ВЫВОД**

- Средние запасы в первом и третьем регионах почти совпадают и составляют 92,5 и 95 тыс. баррелей соотвественно.
- Значения RMSE моделей в этих регионах также похожи 37,5 и 40 тыс. баррелей. Это говорит о том, что разброс запасов в этих регионах достаточно большой. То есть с одной скважины можно получить как 50 тыс., так и 130 тыс. баррелей нефти.
- Во втором регионе средний запас предсказанного сырья значительно меньше, чем в других регионах. Однако, RMSE близко к 0, то есть все скважины имеют достаточно стабильный запас.

## Подготовка к расчёту прибыли

Для подсчёта необходимого объёма сырья для безубыточности разделим бюджет на количество разрабатываемых скважин и на доход с одной скважины.

In [20]:
BUDGET = 10**10
INCOME_PER_ONE = 450*(10**3)
AMOUNT = 200
VOLUME_PER_ONE = BUDGET/AMOUNT/INCOME_PER_ONE

print('Объём сырья с одной скважины для безубыточности:', round(VOLUME_PER_ONE, 2), 'тыс. баррелей')

Объём сырья с одной скважины для безубыточности: 111.11 тыс. баррелей


**ВЫВОД:**
- Во всех регионах предсказанные средние запасы одной скважины меньше, чем необходимое количество для безубыточности.
- Если учитывать разброс, то такие объемы скважин могут быть в первом и третьем регионах.
- Во втором регионе среднее значение предсказанных запасов в 1,5 раза меньше необходимого.

## Расчёт прибыли и рисков 

Напишем функцию для определения прибыли, которая на вход получает предсказания модели, правильные ответы, а также количество скважин, которые будут разрабатываться. Возвращает функция прибыль, учитывая заложенный расход на разработку месторождения. Далее при помощи этой функции и техники `Bootstrap` расчитаем среднюю прибыль и риски для каждого региона.

In [21]:
def profit(target, predictions, count):
    pred_sorted = predictions.sort_values(ascending=False) # сортируем предсказания по убыванию
    selected = target[pred_sorted.index][:count] # выбираем правильные ответы, соответствующие максимальным предсказаниям
    profit = income_per_one * selected.sum()- BUDGET 
    return profit

Так как во втором пункте мы не выделяли в отдельные переменные предсказания и правильные ответы для каждого региона, воспользуемся тем же кодом, но добавим к нему расчёт средней прибыли и 95%-го доверительного интервала для каждого региона.

In [33]:
state = np.random.RandomState(12345)

for index, i in enumerate(datas):
    features = i.drop(['id', 'product'], axis=1)
    target = i['product']
    # отделяем 25% данных для валидационной выборки
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    
    # сбрасываем индексы валидационной выборки для нужной размерности
    target_valid = target_valid.reset_index(drop=True)
    features_valid = features_valid.reset_index(drop=True)

    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    
    predictions = pd.Series(predictions) # преобразовываем массив в Series

    values = [] # создаем пустой список, куда занесем значения прибыли
    
    # применяем технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли
    for i in range(1000):
        target_subsample = target_valid.sample(500, replace=True, random_state=state) 
        pred_subsample = predictions[target_subsample.index]
        values.append(profit(target_subsample, pred_subsample, AMOUNT))

    values = pd.Series(values) # преобразовываем массив в Series
    
    mean = values.mean() # находим среднее значение для всех средних
    lower = values.quantile(q=0.025) # находим 2,5%-ый квантиль
    upper = values.quantile(q=0.975) # находим 97,5%-ый квантиль
    loss = (values < 0).sum()/len(values) # находим риск убытка как отношение количества значений с отрицательной прибыли на все значения
    
    print('Средняя прибыль в регионе №', index+1, ':', round(mean),
          '\n95%-ый доверительный интервал прибыли:', '(', round(lower), ';', round(upper),')', 
          '\n'f'Риск убытков: {loss:.1%}',
         '\n')

Средняя прибыль в регионе № 1 : 425938527 
95%-ый доверительный интервал прибыли: ( -102090095 ; 947976353 ) 
Риск убытков: 6.0% 

Средняя прибыль в регионе № 2 : 518259494 
95%-ый доверительный интервал прибыли: ( 128123231 ; 953612982 ) 
Риск убытков: 0.3% 

Средняя прибыль в регионе № 3 : 420194005 
95%-ый доверительный интервал прибыли: ( -115852609 ; 989629940 ) 
Риск убытков: 6.2% 



## Вывод

- Средняя прибыль в первом и третьем регионах почти одинаковая около 420 млн руб. А во втором регионе средняя прибыль значительно выше и составляет 518 млн руб.
- Риск получить убытки в первом и в первом и третьем регионах около 6%. Для второго региона риск убытков всего 0.3%. 
- Для разработки скважин рекомендовано выбрать регион №2. Предсказанная суммарная средняя прибыль со всех скважин значительно больше, чем в других регионах. А вероятность рисков убытка меньше 2,5%.